In [73]:
import json
import re
import nltk

In [74]:
dataFile = open('formatted_data.json', 'r')

In [75]:
data = []

In [76]:
for line in dataFile:
    data.append(json.loads(line))

In [77]:
types = []
posts = []

In [78]:
for i in range(1, len(data)):
    types.append(data[i]['Type'])
    posts.append(data[i]['Post'])

In [79]:
# import random
# SEED = 673

# random.seed(SEED)
# random.shuffle(types)
# random.shuffle(posts)

In [80]:
def vectorize_post_data(posts):
    for index in range(0, len(posts)):
        posts[index] = posts[index].split("|||")
        
    return posts
        
posts = vectorize_post_data(posts)

In [81]:
def remove_hyperTextLinksFromPosts(posts):
    for index in range(0, len(posts)):
        usable_post = [post for post in posts[index] if not re.search(r'^(.)*http(.)*$', post)]
        posts[index] = usable_post
    
    return posts
        
posts = remove_hyperTextLinksFromPosts(posts)

In [ ]:
def remove_MBTIClassifiersFromPosts(posts):
    MBTI_regex = r'[\w]*(i|e)(s|n)(f|t)(p|j)[\w]*'
    for i in range(0, len(posts)):
        for j in range(0, len(posts[i])):
            posts[i][j] = re.sub(MBTI_regex, ' ', posts[i][j], flags=re.IGNORECASE)
        
    return posts
    
posts = remove_MBTIClassifiersFromPosts(posts)

In [95]:
from nltk.tokenize import RegexpTokenizer

def tokenize_posts(posts):
    tokenizer = RegexpTokenizer(r'\w+')
    
    for i in range(0, len(posts)):
        user_words = []
        for j in range(0, len(posts[i])):
            post = tokenizer.tokenize(posts[i][j])
            for word in post:
                user_words.append(word)
        posts[i] = user_words
    
    return posts

posts = tokenize_posts(posts)

In [13]:
vectorized_posts5 = []

def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def remove_numbers_in_posts(posts):
    counter = 0
    for index in range(0, len(posts)):
        words = [word.lower() for word in posts[index] if not isInt(word)]
        vectorized_posts5.append(words)
        counter += 1
        

remove_numbers_in_posts(vectorized_posts4)

In [14]:
all_words_list = []

for user in vectorized_posts5:
    for word in user:
        all_words_list.append(word)


In [15]:
from collections import Counter

In [16]:
freq_list = Counter(all_words_list)
dictionary = freq_list.most_common(10000)

In [17]:
dictionary = list(zip(*dictionary))[0]

In [18]:
nums = range(0, 10000)
word_int = dict(zip(dictionary, nums))

In [19]:
x_vals = []

for user in vectorized_posts5:
    x_vals.append([word_int[x] for x in user if x in word_int.keys()])

In [20]:
intro_extro = [word[1:2] for word in types[1:len(types)]]
bin_intro_extro = []
for letter in intro_extro:
    if (letter == 'S'):
        bin_intro_extro.append(0)
    else:
        bin_intro_extro.append(1)

In [21]:
import numpy as np
x = np.array(x_vals)
test_data = x[:1500]
train_data = x[1500:]

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [22]:
x_train.size

71750000

In [23]:
from keras.utils.np_utils import to_categorical

y = np.asarray(bin_intro_extro).astype('float32')
y_test = y[:1500]
y_train = y[1500:]

Using TensorFlow backend.


In [24]:
len(y_train)

7175

In [25]:
x_val = x_train[:1500]
x_partial_train = x_train[1500:]

y_val = y_train[:1500]
y_partial_train = y_train[1500:]

In [26]:
len(y_partial_train)

5675

In [27]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
history = model.fit(x_partial_train, y_partial_train, epochs = 20, batch_size = 512, validation_data=(x_val, y_val))

Train on 5675 samples, validate on 1500 samples
Epoch 1/20
5675/5675 [==============================] - 1s 170us/step - loss: 0.4414 - acc: 0.8169 - val_loss: 0.4616 - val_acc: 0.8520
Epoch 2/20
5675/5675 [==============================] - 1s 107us/step - loss: 0.3798 - acc: 0.8624 - val_loss: 0.5084 - val_acc: 0.8520
Epoch 3/20
5675/5675 [==============================] - 1s 109us/step - loss: 0.3546 - acc: 0.8624 - val_loss: 0.4363 - val_acc: 0.8520
Epoch 4/20
5675/5675 [==============================] - 1s 104us/step - loss: 0.3310 - acc: 0.8624 - val_loss: 0.4338 - val_acc: 0.8520
Epoch 5/20
5675/5675 [==============================] - 1s 110us/step - loss: 0.2860 - acc: 0.8626 - val_loss: 0.4793 - val_acc: 0.8520
Epoch 6/20
5675/5675 [==============================] - 1s 111us/step - loss: 0.2621 - acc: 0.8689 - val_loss: 0.4489 - val_acc: 0.8513
Epoch 7/20
5675/5675 [==============================] - 1s 104us/step - loss: 0.2428 - acc: 0.8925 - val_loss: 0.5269 - val_acc: 0.8513


In [29]:
results = model.evaluate(x_test, y_test)
results

1500/1500 [==============================] - 0s 103us/step


[0.7880286078453064, 0.8459999995231628]

In [30]:
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [31]:
model.fit(x_train, y_train, epochs = 13, batch_size = 512)

Epoch 1/13
7175/7175 [==============================] - 1s 113us/step - loss: 0.4447 - acc: 0.8513
Epoch 2/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3868 - acc: 0.8602
Epoch 3/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3666 - acc: 0.8602
Epoch 4/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3202 - acc: 0.8602
Epoch 5/13
7175/7175 [==============================] - 1s 85us/step - loss: 0.3104 - acc: 0.8652
Epoch 6/13
7175/7175 [==============================] - 1s 82us/step - loss: 0.2849 - acc: 0.8711
Epoch 7/13
7175/7175 [==============================] - 1s 88us/step - loss: 0.2538 - acc: 0.8966
Epoch 8/13
7175/7175 [==============================] - 1s 105us/step - loss: 0.2228 - acc: 0.9111
Epoch 9/13
7175/7175 [==============================] - 1s 83us/step - loss: 0.1994 - acc: 0.9239
Epoch 10/13
7175/7175 [==============================] - 1s 83us/step - loss: 0.1762 - acc: 0.9338
Epoch 11/13
7175/

In [32]:
results = model.evaluate(x_test, y_test)
results

1500/1500 [==============================] - 0s 162us/step


[0.6722889490922292, 0.8646666661898295]